In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

1115394/1115394 [==============================] - 1s 0us/step


In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
text = text[30000:800000]

In [5]:
characters = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [6]:
SEQ_LENGTH = 40
STEP_SIZE = 3
sentences = []
next_char = []

In [7]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

In [8]:
bool_array = np.array([True, False, True], dtype=np.bool_)
x = np.zeros((len(sentences), SEQ_LENGTH,len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences),len(characters)), dtype=np.bool_)
for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

In [9]:
model = Sequential()
model.add(LSTM(128,input_shape=(SEQ_LENGTH,len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

In [10]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.01))
model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
1003/1003 [==============================] - 205s 202ms/step - loss: 2.5907
Epoch 2/4
1003/1003 [==============================] - 203s 203ms/step - loss: 2.2311
Epoch 3/4
1003/1003 [==============================] - 203s 202ms/step - loss: 2.1050
Epoch 4/4
1003/1003 [==============================] - 202s 201ms/step - loss: 2.0139


In [11]:
def sample(preds, temperature=100):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [13]:
def generate_mood_text(mood, length, temperature):

  mood_dict = {
    "happy": ["joyful", "cheerful", "optimistic", "excited"],
    "sad": ["sorrowful", "depressed", "melancholy", "tearful"],
    "angry": ["furious", "enraged", "indignant", "hostile"],
    "calm": ["peaceful", "serene", "tranquil", "composed"],
  }

  mood_words = mood_dict.get(mood.lower(), [])

  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence

  for i in range(length):
    x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, char in enumerate(sentence):
      x_predictions[0, t, char_to_index[char]] = 1

    predictions = model.predict(x_predictions, verbose=0)[0]

    filtered_predictions = [index for index, prediction in enumerate(predictions) if index_to_char[index] in mood_words]


    if len(filtered_predictions) > 0:
      next_index = random.choice(filtered_predictions)
    else:
      next_index = sample(predictions, temperature)

    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character

  return generated



In [14]:
print("----------0.3--------")
print(generate_mood_text("love", 300, 0.3))

----------0.3--------
mmands.

gloucester:

clarence:

lady grome the me and the wert the say
that she whe heave she werd sear,
and the werd the beath the will when,
the prave the sore what the wall the king
and his the geads.

corines:
and ho she me the mere the ward the pree.

herr comen:
the werll the some the beath the with heve
the beath me for dears enes
